In [ ]:
from typing import List

from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine_v1 as discoveryengine


def search_sample(
    project_id: str,
    location: str,
    engine_id: str,
    search_query: str,
) -> List[discoveryengine.SearchResponse]:
    
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    # Create a client
    client = discoveryengine.SearchServiceClient(client_options=client_options)

    # The full resource name of the search app serving config
    serving_config = f"projects/{project_id}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/default_config"

    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        page_size=10,
        # content_search_spec=content_search_spec,
        query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
            condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        ),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
    )

    response = client.search(request)
    result = next(response.pages)
    search_results = result.results
    print(search_results) 
    all_search_result = []
    
    
    for i,result1 in enumerate(search_results,1):
        result_doc = result1.document.struct_data
        search_result = {}
        search_result['product_name'] = result_doc['product_name']
        search_result['product_url'] = result_doc['product_url']

        all_search_result.append(search_result)

    return all_search_result



project_id = "amw-dna-coe-working-ds-dev"
location = "global"                   # Values: "global", "us", "eu"
engine_id = "product-search-app_1732269184635"
search_query = "What are some recommended hair products for men?"
res2 = search_sample(
    project_id=project_id,
    location=location,
    engine_id=engine_id,
    search_query=search_query
)

[id: "ee0524935e4e443d2f110950dfa8c4c0"
document {
  name: "projects/223619550913/locations/global/collections/default_collection/dataStores/product-index-data_1732267482383/branches/0/documents/ee0524935e4e443d2f110950dfa8c4c0"
  id: "ee0524935e4e443d2f110950dfa8c4c0"
  struct_data {
    fields {
      key: "product_url"
      value {
        string_value: "https://www.amway.com/en_US/Body-Blends-for-Men-Hair-Putty-%E2%80%93-Xtreme-Live-on-the-Edge-p-761531"
      }
    }
    fields {
      key: "product_name"
      value {
        string_value: "Body Blends for Men Hair Putty – Xtreme Live on the Edge"
      }
    }
    fields {
      key: "product_id"
      value {
        string_value: "761531"
      }
    }
    fields {
      key: "product_description"
      value {
        string_value: "<p>100 mL (3.38 oz.)</p><p><strong>Maximum control. Effortless manipulation</strong></p><p>Designed with a powerful and firm holding technology, the ability to create and control any hairstyle is

In [11]:
res2

[{'product_name': 'Body Blends for Men Hair Putty – Xtreme Live on the Edge',
  'product_url': 'https://www.amway.com/en_US/Body-Blends-for-Men-Hair-Putty-%E2%80%93-Xtreme-Live-on-the-Edge-p-761531'},
 {'product_name': 'Artistry™ Men System',
  'product_url': 'https://www.amway.com/en_US/Artistry%E2%84%A2-Men-System-p-118205'},
 {'product_name': 'Satinique™ Smooth Moisture Shampoo – 750 mL',
  'product_url': 'https://www.amway.com/en_US/Satinique%E2%84%A2-Smooth-Moisture-Shampoo-%E2%80%93-750-mL-p-110656'},
 {'product_name': 'Artistry Men Unknown&trade; Hair and Body Wash',
  'product_url': 'https://www.amway.com/en_US/Artistry-Men-Unknown%26trade%3B-Hair-and-Body-Wash-p-120550D'},
 {'product_name': 'Satinique™ Overnight Repair Treatment',
  'product_url': 'https://www.amway.com/en_US/Satinique%E2%84%A2-Overnight-Repair-Treatment-p-110677'},
 {'product_name': 'Satinique™ Hair Oil Serum',
  'product_url': 'https://www.amway.com/en_US/Satinique%E2%84%A2-Hair-Oil-Serum-p-120467'},
 {'prod

In [43]:
import numpy as np 
def compute_similarity(query_embedding, doc_embedding):
    """Compute cosine similarity between embeddings."""
    return np.dot(query_embedding, doc_embedding) / (
        np.linalg.norm(query_embedding) * np.linalg.norm(doc_embedding)
    )
from typing import Dict, List, Optional, Literal, Annotated, Sequence
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

def rank_documents(
     docs: List[Dict], query: str
) -> List[Dict]:
    """Rank documents based on relevance to query."""
    query_embedding = model.encode([query])[0]

    doc_similarities = []
    for doc in docs:
        content = doc["product_description"]
        doc_embedding = model.encode([content])[0]
        similarity = compute_similarity(query_embedding, doc_embedding)
        doc_similarities.append((similarity, doc))

    threshold = 0.3
    ranked_docs = [
        doc
        for score, doc in sorted(doc_similarities, key=lambda x: x[0], reverse=True)
        if score > threshold
    ]
    return ranked_docs



In [44]:
ranked_docs = rank_documents(res2,"what is the importance of clean water")
ranked_docs[:5]

[{'index': 10,
  'product_id': '401049S',
  'product_name': 'The Home Book 2024–2025, 10-pack – Spanish',
  'product_description': '<p>10 Catalogs</p>  <p><strong>Easily shop all our healthy home solutions.</strong></p>  <p>The&nbsp;<em>Home Book</em>&nbsp;is where you&rsquo;ll find safe and effective cleaning products that are better for you and the environment, plus our other home-support brands.</p>  <p>The Amway Home&trade; brand makes it easy to care for your home with safe and effective surface, dish and laundry cleaners and disinfectants. Surface cleaners are made better for you and your home with proven, plant-derived ingredients that are concentrated so you use less to clean more. Detergents are natural dirt lifters that offer super cleaning power to fight stains. It&rsquo;s cleaning power you can trust.</p>  <p>You&rsquo;ll be able to create wholesome, healthy meals with our easy-to-clean, quality iCook&trade; Cookware and Knifeware. The eSpring&trade; Water Purification Syst